## Extract feature function

In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

def extract_features(file_name, type="mfcc"):
    # Load audio file
    file_path = "/Users/Licious/project/unipd/hda_dataset/audio/"+ file_name
    y, sr = librosa.load(file_path, sr=None)

    # Extract MFCCs (Mel-Frequency Cepstral Coefficients)
    if type=="mfcc":
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        return mfccs
    elif type=="chroma":
    # print(mfccs)
    # Extract Chroma feature
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        return chroma

## Train_test_spit

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import pandas as pd

# Function to load and preprocess the dataset
def load_and_preprocess_data():
    # Assume you have a list of file paths and corresponding labels
    meta = pd.read_csv("/Users/Licious/project/unipd/hda_dataset/meta/esc50.csv")

    file_paths = meta["filename"].to_list()
    labels = meta["target"].to_list()

    # Extract features for each audio file
    feature_mfcc = [extract_features(file_path, type="mfcc") for file_path in file_paths]
    features_chroma = [extract_features(file_path, type="chroma") for file_path in file_paths]

    # Convert lists to numpy arrays
    feature_mfcc = np.array(feature_mfcc)
    features_chroma = np.array(features_chroma)
    labels = np.array(labels)

    return feature_mfcc,features_chroma, labels

num_classes = 50
# Load and preprocess the data
# dataset_path = "path/to/your/dataset"
print("Start Feature extraction")
feature_mfcc, features_chroma, labels = load_and_preprocess_data()
print("End Feature extraction")


# Split the data into training and testing sets
X_train_mfcc, X_test_mfcc, X_train_chroma, X_test_chroma, y_train, y_test = train_test_split(feature_mfcc, features_chroma, labels, test_size=0.2, random_state=42)


Start Feature extraction


/Users/Licious/project/DataPlatform/pydev/lib/python3.9/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


End Feature extraction


In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import numpy as np

# Assuming you have feature_mfcc, features_chroma, and labels as your original datasets

# Use train_test_split to split the data into training and testing sets
# X_train_mfcc, X_test_mfcc, X_train_chroma, X_test_chroma, y_train, y_test = train_test_split(
#     feature_mfcc, features_chroma, labels, test_size=0.2, random_state=42
# )

# Build the model
mfcc_input = layers.Input(shape=(X_train_mfcc.shape[1], X_train_mfcc.shape[2],1), name='mfcc_input')
mfcc_conv1 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='valid')(mfcc_input)
mfcc_pool1 = layers.MaxPooling2D(pool_size=(2, 2))(mfcc_conv1)
mfcc_batch_norm1 = layers.BatchNormalization()(mfcc_pool1)
mfcc_conv2 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='valid')(mfcc_batch_norm1)
mfcc_pool2 = layers.MaxPooling2D(pool_size=(2, 2))(mfcc_conv2)
mfcc_batch_norm2 = layers.BatchNormalization()(mfcc_pool2)

# chroma_input = layers.Input(shape=(X_train_chroma.shape[1], X_train_chroma.shape[2],1), name='chroma_input')
# chroma_conv1 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='valid')(chroma_input)
# chroma_pool1 = layers.MaxPooling2D(pool_size=(2, 2))(chroma_conv1)
# chroma_batch_norm1 = layers.BatchNormalization()(chroma_pool1)
# chroma_conv2 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='valid')(chroma_batch_norm1)
# chroma_pool2 = layers.MaxPooling2D(pool_size=(2, 2))(chroma_conv2)
# chroma_batch_norm2 = layers.BatchNormalization()(chroma_pool2)

# merged = layers.Concatenate()([mfcc_batch_norm2, chroma_batch_norm2])
merged = layers.Concatenate()([mfcc_batch_norm2])

reshaped = layers.Reshape((106, 64))(merged)
lstm1 = layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer='l2')(reshaped)
lstm2 = layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer='l2')(lstm1)
lstm3 = layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer='l2')(lstm2)

flat = layers.Flatten()(lstm3)
dense1 = layers.Dense(128, activation='relu')(flat)
dropout = layers.Dropout(0.5)(dense1)

num_classes = len(np.unique(labels))
output = layers.Dense(num_classes, activation='softmax', name='output')(dropout)

model = models.Model(inputs=[mfcc_input], outputs=output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit({'mfcc_input': X_train_mfcc, 'chroma_input': X_train_chroma}, y_train, epochs=10, batch_size=64,
          validation_data=({'mfcc_input': X_test_mfcc, 'chroma_input': X_test_chroma}, y_test))

test_loss, test_acc = model.evaluate({'mfcc_input': X_test_mfcc, 'chroma_input': X_test_chroma}, y_test)
print("Test Accuracy:", test_acc)


Epoch 1/10


/Users/Licious/project/DataPlatform/pydev/lib/python3.9/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['chroma_input'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


25/25 [==============================] - 9s 291ms/step - loss: 6.1016 - accuracy: 0.0981 - val_loss: 7.0395 - val_accuracy: 0.0275
Epoch 2/10
25/25 [==============================] - 6s 259ms/step - loss: 4.8103 - accuracy: 0.2050 - val_loss: 6.3074 - val_accuracy: 0.0475
Epoch 3/10
25/25 [==============================] - 6s 255ms/step - loss: 4.0556 - accuracy: 0.2713 - val_loss: 5.8585 - val_accuracy: 0.0750
Epoch 4/10
25/25 [==============================] - 6s 260ms/step - loss: 3.6017 - accuracy: 0.3094 - val_loss: 4.6532 - val_accuracy: 0.1400
Epoch 5/10
25/25 [==============================] - 6s 256ms/step - loss: 3.2586 - accuracy: 0.3431 - val_loss: 4.3762 - val_accuracy: 0.1800
Epoch 6/10
25/25 [==============================] - 6s 259ms/step - loss: 2.9232 - accuracy: 0.4087 - val_loss: 3.5980 - val_accuracy: 0.2500
Epoch 7/10
25/25 [==============================] - 7s 266ms/step - loss: 2.7513 - accuracy: 0.4156 - val_loss: 3.4075 - val_accuracy: 0.3025
Epoch 8/10
25/25 

In [ ]:
X_train = X_train_mfcc.reshape(X_train_mfcc.shape[0], X_train_mfcc.shape[1], X_train_mfcc.shape[2], 1)
X_test = X_test_mfcc.reshape(X_test_mfcc.shape[0], X_test_mfcc.shape[1], X_test_mfcc.shape[2], 1)

# CRNN Model
model = models.Sequential()

# Convolutional layers for feature extraction
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1), padding='valid'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='valid'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Reshape for recurrent layers

model.add(layers.Reshape((106, 64)))
# model.add(layers.LSTM(64, return_sequences=True))
# Example: Add dropout to LSTM layers
model.add(layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer='l2'))
# Example: Add dropout to LSTM layers
model.add(layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer='l2'))
# Example: Add dropout to LSTM layers
model.add(layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer='l2'))

# model.add(layers.LSTM(64, return_sequences=True))
# model.add(layers.LSTM(64, return_sequences=True))

# Flatten and Dense layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))

# Output layer
num_classes = len(np.unique(labels))
model.add(layers.Dense(num_classes, activation='softmax'))


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)